In [1]:
import requests
import re
import json
import pandas as pd
from pandas import json_normalize
import time
from datetime import timezone
import numpy as np
from tqdm import tqdm
from pycoingecko import CoinGeckoAPI
import datetime
cg = CoinGeckoAPI()

In [11]:
def getCoin(coin, from_time, to_time):
    df = cg.get_coin_market_chart_range_by_id(id=coin, vs_currency='usd', from_timestamp=from_time, to_timestamp=to_time)

    df_market_caps = pd.DataFrame(df["market_caps"]).rename(columns={0:"date",1:coin+"_market_cap"})
    df_prices = pd.DataFrame(df["prices"]).rename(columns={0:"date",1:coin+"_price"})
    df_volumes = pd.DataFrame(df["total_volumes"]).rename(columns={0:"date",1:coin+"_volume"})

    df_market_caps['date'] = pd.to_datetime(df_market_caps['date'], unit='ms').map(lambda x: x.strftime('%Y-%m-%d'))
    df_prices['date'] = pd.to_datetime(df_prices['date'], unit='ms').map(lambda x: x.strftime('%Y-%m-%d'))
    df_volumes['date'] = pd.to_datetime(df_volumes['date'], unit='ms').map(lambda x: x.strftime('%Y-%m-%d'))

    
    return df_market_caps, df_prices, df_volumes


#returns price history of multiple coins
def getCoins(coins, from_time, to_time):
    mc, p, v = getCoin(coins[0], from_time, to_time)
    
    df_market_caps = mc
    df_prices = p
    df_volumes = v
     
    for coin in tqdm(coins[1:]):
        try:
            mc, p, v = getCoin(coin, from_time, to_time)
            df_market_caps = df_market_caps.merge(mc, how="outer", on="date")
            df_prices = df_prices.merge(p, how="outer", on="date")
            df_volumes = df_volumes.merge(v, how="outer", on="date")
        except:
            pass
        
    df_market_caps.set_index("date", inplace=True)
    df_prices.set_index("date", inplace=True)
    df_volumes.set_index("date", inplace=True)

    return df_market_caps, df_prices, df_volumes

def getCoinNames(count):
    pages = count // 250
    extra = count % 250
    names = []
    
    for i in range(1,pages+1):
        c = cg.get_coins(vs_currency='USD', per_page=250,page=i)
        names = names + [c[i]["id"] for i in range(250)]
    
    c = cg.get_coins(vs_currency='USD', per_page=250,page=pages+1)
    names = names + [c[i]["id"] for i in range(extra)]
    
    return names
    

In [3]:
date1 = datetime.datetime(2017, 12, 31)
date2 = datetime.datetime(2023, 1, 15)

In [14]:
names = getCoinNames(500)

In [15]:
market_caps, prices, volumes = getCoins(names, date1.timestamp(), date2.timestamp())

100%|█████████████████████████████████████████████████████████████████████████████| 499/499 [10:28:06<00:00, 75.52s/it]


In [16]:
market_caps.to_csv("data/new/market_caps.csv")
prices.to_csv("data/new/prices.csv")
volumes.to_csv("data/new/volumes.csv")